## Librerías :

In [2]:
import tensorflow as tf
from tensorflow import keras

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.metrics import *
%matplotlib inline

## Data:


In [26]:
data1 = pd.read_csv('../files/driver_imgs_list.csv')
data1[data1["img"]=="img_5.jpg"]

,subject,classname,img
19762,p066,c3,img_5.jpg
